In [10]:
import torch
import torchvision
import torchvision.transforms.v2 as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cnn import *
from ffn import *

In [11]:
OUTPUT_DIM = 10
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28
INPUT_DIM = IMAGE_HEIGHT * IMAGE_WIDTH

In [12]:
transform = transforms.Compose([                            # Use transforms to convert images to tensors and normalize them
    transforms.ToTensor(),                                  # convert images to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])             # Common method for grayscale images
])

batch_size = 64 # 32–128

/Users/griffinmichalak/Desktop/code/fall25/intro-ai/PA3-1/venv/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [13]:
trainset = torchvision.datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [14]:
feedforward_net = FF_Net()
# conv_net = Conv_Net()

criterion = nn.CrossEntropyLoss() # loss function

optimizer_ffn = optim.Adam(feedforward_net.parameters(), lr=LEARNING_RATE)

# Training

In [15]:
num_epochs_ffn = NUM_EPOCHS

for epoch in range(num_epochs_ffn):  # loop over the dataset multiple times
    running_loss_ffn = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Flatten inputs for ffn
        ''' flattening within FF_Net.forward() '''

        # zero the parameter gradients
        optimizer_ffn.zero_grad()

        # forward + backward + optimize
        outputs = feedforward_net(inputs)  # forward pass happens here
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ffn.step()
        running_loss_ffn += loss.item()

    print(f"Training loss: {running_loss_ffn}")

print('Finished Training')

torch.save(feedforward_net.state_dict(), 'ffn.pth')  # Saves model file (upload with submission)

Training loss: 453.98809637129307


KeyboardInterrupt: 

# Evaluation

In [16]:
'''

PART 6:
Evalute your models! Accuracy should be greater or equal to 80% for both models.

Code to load saved weights commented out below - may be useful for debugging.

'''

# feedforward_net.load_state_dict(torch.load('ffn.pth'))
# conv_net.load_state_dict(torch.load('cnn.pth'))

correct_ffn = 0
total_ffn = 0

with torch.no_grad():           # since we're not training, we don't need to calculate the gradients for our outputs
    for data in testloader:
        images, labels = data
        outputs = feedforward_net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_ffn = correct_ffn + 1
            total_ffn = total_ffn + 1

print('Accuracy for feedforward network: ', correct_ffn/total_ffn)
# print('Accuracy for convolutional network: ', correct_cnn/total_cnn)

Accuracy for feedforward network:  0.8663
